라이브러리

In [ ]:
!pip install xgboost
!pip install catboost
!pip install optuna
!pip install pycaret

In [ ]:
import pandas as pd
import numpy as np

import xgboost as xgb
from catboost import CatBoostRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import ExtraTreesRegressor
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.preprocessing import PolynomialFeatures
from sklearn.ensemble import VotingRegressor
import optuna
from optuna import Trial, visualization
import time
from pycaret.regression import *

전처리(사계절분리)

In [ ]:
# 원본데이터
train = pd.read_csv('/content/drive/MyDrive/surface_tp_train.csv')
test=pd.read_csv('/content/drive/MyDrive/surface_tp_test.csv')
train = train.iloc[:,1:]
test = test.iloc[:,1:]

# 컬럼명 정리
col = ['stn','year','mmddhh','ta','td','hm','ws','rn','re','ww','ts','si','ss','sn']
col2 = ['stn','year','mmddhh','ta','td','hm','ws','rn','re','ww','si','ss','sn']
train.columns = col
test.columns = col2

In [ ]:
# 날짜에 따른 사계절 분리

spring_train = pd.concat(
    [
        train[train["surface_tp_train.mmddhh"].astype(str).str.startswith("2")],
        train[train["surface_tp_train.mmddhh"].astype(str).str.startswith("3")],
        train[train["surface_tp_train.mmddhh"].astype(str).str.startswith("4")],
    ]
)
summer_train = pd.concat(
    [
        train[train["surface_tp_train.mmddhh"].astype(str).str.startswith("5")],
        train[train["surface_tp_train.mmddhh"].astype(str).str.startswith("6")],
        train[train["surface_tp_train.mmddhh"].astype(str).str.startswith("7")],
    ]
)
fall_train = pd.concat(
    [
        train[train["surface_tp_train.mmddhh"].astype(str).str.startswith("8")],
        train[train["surface_tp_train.mmddhh"].astype(str).str.startswith("9")],
        train[
            train["surface_tp_train.mmddhh"].astype(str).str.startswith("10")
            & (train["surface_tp_train.mmddhh"].astype(str).str.len() == 6)
        ],
    ]
)
winter_train = pd.concat(
    [
        train[
            train["surface_tp_train.mmddhh"].astype(str).str.startswith("11")
            & (train["surface_tp_train.mmddhh"].astype(str).str.len() == 6)
        ],
        train[
            train["surface_tp_train.mmddhh"].astype(str).str.startswith("12")
            & (train["surface_tp_train.mmddhh"].astype(str).str.len() == 6)
        ],
        train[
            train["surface_tp_train.mmddhh"].astype(str).str.startswith("1")
            & (train["surface_tp_train.mmddhh"].astype(str).str.len() == 5)
        ],
    ]
)

spring_test = pd.concat(
    [
        test[test["surface_tp_test.mmddhh"].astype(str).str.startswith("2")],
        test[test["surface_tp_test.mmddhh"].astype(str).str.startswith("3")],
        test[test["surface_tp_test.mmddhh"].astype(str).str.startswith("4")],
    ]
)
summer_test = pd.concat(
    [
        test[test["surface_tp_test.mmddhh"].astype(str).str.startswith("5")],
        test[test["surface_tp_test.mmddhh"].astype(str).str.startswith("6")],
        test[test["surface_tp_test.mmddhh"].astype(str).str.startswith("7")],
    ]
)
fall_test = pd.concat(
    [
        test[test["surface_tp_test.mmddhh"].astype(str).str.startswith("8")],
        test[test["surface_tp_test.mmddhh"].astype(str).str.startswith("9")],
        test[
            test["surface_tp_test.mmddhh"].astype(str).str.startswith("10")
            & (test["surface_tp_test.mmddhh"].astype(str).str.len() == 6)
        ],
    ]
)
winter_test = pd.concat(
    [
        test[
            test["surface_tp_test.mmddhh"].astype(str).str.startswith("11")
            & (test["surface_tp_test.mmddhh"].astype(str).str.len() == 6)
        ],
        test[
            test["surface_tp_test.mmddhh"].astype(str).str.startswith("12")
            & (test["surface_tp_test.mmddhh"].astype(str).str.len() == 6)
        ],
        test[
            test["surface_tp_test.mmddhh"].astype(str).str.startswith("1")
            & (test["surface_tp_test.mmddhh"].astype(str).str.len() == 5)
        ],
    ]
)

전처리(결측치 대체)

In [ ]:
# 결측치 유무 확인 코드

missing_values_2 = [-99, -99.9, -999, -99.90]

for column in train.columns:
    # 각 컬럼에서 결측치가 있는지 확인
    missing_values = train[column].isin(missing_values_2).sum()

    if missing_values > 0:
        print(f"{column}: {missing_values} missing values")
    else:
        print(f"{column}: No missing values")

In [ ]:
# 평균,중위수,보간법으로 결측치를 대체하는 함수 V1(Q),V2(Q),V3(Q)

In [ ]:
def V1(Q):
  df2 = df.replace([-99, -99.9, -999], np.nan)
  df2[Q][df2[df2[Q].isna()].index] = np.nanmean(df2[Q]) # 컬럼Q nan값 평균값 대체
  df2 = df2.fillna(0)   # 나머지 결측치 0으로 변경

  # 필요없는 컬럼들 drop
  to_drops = [
      "year"]
  # X, y로 분리
  X = df2.drop(["ts"], axis='columns')
  y = df2["ts"]

  # object => LabelEncoder, !object => StandardScaler
  object_cols = []
  non_object_cols = []
  for col in X.columns:
      if X[col].dtype == 'object':
          object_cols.append(col)
      else:
          non_object_cols.append(col)

  # LabelEncoder 적용
  le = LabelEncoder()
  for col in object_cols:
      X[col] = le.fit_transform(X[col])

  # StandardScaler 적용
  ss = StandardScaler()
  for col in non_object_cols:
      X[col] = ss.fit_transform(X[[col]])

  # lightGBM + 교차검증 + MAE(평가지표)
  # LightGBM 모델 생성
  model = LGBMRegressor(random_state=42)

  # 교차 검증 설정
  kf = KFold(n_splits=5, random_state=42, shuffle=True)
  scoring = make_scorer(mean_absolute_error, greater_is_better=True)
  cv_scores = cross_val_score(model, X, y, cv=kf, scoring=scoring)

  # 전체 교차 검증 결과 출력
  print(f"평균대체{Q}")
  print(f"전체 MAE: {cv_scores}")
  print(f"평균 MAE: {np.mean(cv_scores)}")
  print(f"MAE 표준편차: {np.std(cv_scores)}")

In [ ]:
def V2(Q):
  df2 = df.replace([-99, -99.9, -999], np.nan)
  df2[Q][df2[df2[Q].isna()].index] = np.nanmedian(df2[Q])    # ta컬럼 nan값 중위수로 대체
  df2 = df2.fillna(0)   # 나머지 결측치 0으로 변경

  # @@@데이터 로드@@@ "eval(input(...))"을 입력 변수로 수정해주세요
  dfm = df2

  # 필요없는 컬럼들 drop
  to_drops = [
      "year",
  ]
  for to_drop in to_drops:
      if to_drop in dfm.columns:
          dfm = dfm.drop(to_drop, axis='columns')

  # X, y로 분리
  X = dfm.drop(["ts"], axis='columns')
  y = dfm["ts"]

  # object => LabelEncoder, !object => StandardScaler
  object_cols = []
  non_object_cols = []
  for col in X.columns:
      if X[col].dtype == 'object':
          object_cols.append(col)
      else:
          non_object_cols.append(col)

  # LabelEncoder 적용
  le = LabelEncoder()
  for col in object_cols:
      X[col] = le.fit_transform(X[col])

  # StandardScaler 적용
  ss = StandardScaler()
  for col in non_object_cols:
      X[col] = ss.fit_transform(X[[col]])

  # lightGBM + 교차검증 + MAE(평가지표)
  # LightGBM 모델 생성
  model = LGBMRegressor(random_state=42)

  # 교차 검증 설정
  kf = KFold(n_splits=5, random_state=42, shuffle=True)
  scoring = make_scorer(mean_absolute_error, greater_is_better=True)
  cv_scores = cross_val_score(model, X, y, cv=kf, scoring=scoring)

  # 전체 교차 검증 결과 출력
  print(f"중위수대체{Q}")
  print(f"전체 MAE: {cv_scores}")
  print(f"평균 MAE: {np.mean(cv_scores)}")
  print(f"MAE 표준편차: {np.std(cv_scores)}")

In [ ]:
def V3(Q):
  df2 = df.replace([-99, -99.9, -999], np.nan)
  df2[Q] = df2[Q].interpolate(method='linear', limit_direction='forward', x=df2[df2.corr()[Q].drop(Q).idxmax()])  # ta컬럼 nan값 보간법으로 대체.  df2.corr()['ta'].drop('ta').idxmax(): ta컬럼과 상관관계가 가장 높은 컬럼
  df2=df2.fillna(0)   # 나머지 결측치 0으로 변경

  # @@@데이터 로드@@@ "eval(input(...))"을 입력 변수로 수정해주세요
  dfm = df2

  # 필요없는 컬럼들 drop
  to_drops = [
      "year",
  ]
  for to_drop in to_drops:
      if to_drop in dfm.columns:
          dfm = dfm.drop(to_drop, axis='columns')

  # X, y로 분리
  X = dfm.drop(["ts"], axis='columns')
  y = dfm["ts"]

  # object => LabelEncoder, !object => StandardScaler
  object_cols = []
  non_object_cols = []
  for col in X.columns:
      if X[col].dtype == 'object':
          object_cols.append(col)
      else:
          non_object_cols.append(col)

  # LabelEncoder 적용
  le = LabelEncoder()
  for col in object_cols:
      X[col] = le.fit_transform(X[col])

  # StandardScaler 적용
  ss = StandardScaler()
  for col in non_object_cols:
      X[col] = ss.fit_transform(X[[col]])

  # lightGBM + 교차검증 + MAE(평가지표)
  # LightGBM 모델 생성
  model = LGBMRegressor(random_state=42)

  # 교차 검증 설정
  kf = KFold(n_splits=5, random_state=42, shuffle=True)
  scoring = make_scorer(mean_absolute_error, greater_is_better=True)
  cv_scores = cross_val_score(model, X, y, cv=kf, scoring=scoring)

  # 전체 교차 검증 결과 출력
  print(f"보간법{Q}")
  print(f"전체 MAE: {cv_scores}")
  print(f"평균 MAE: {np.mean(cv_scores)}")
  print(f"MAE 표준편차: {np.std(cv_scores)}")

In [ ]:
# 각 컬럼별 결측치를 대체하는 최상의 방법이 무엇인지 확인하는 for문

col_list = ['ta','td','hm','ws','rn','re','ts','si','ss','sn']
for i in col_list:
  V1(i)
  V2(i)
  V3(i)
  print('-'*50)

In [ ]:
# 피처 엔지니어링
df2['mm'] = df2['mmddhh'] // 10000           # 'mmddhh' 컬럼에서 'mm', 'dd', 'hh' 분리
df2['dd'] = (df2['mmddhh'] // 100) % 100
df2['hh'] = df2['mmddhh'] % 100

In [ ]:
# spring_train 만들기(다른 계절도 동일)

# spring_train
df2 = df.replace([-99, -99.9, -999], np.nan)
df2['ta'][df2[df2['ta'].isna()].index] = np.nanmedian(df2['ta'])
df2['td'][df2[df2['td'].isna()].index] = np.nanmean(df2['td'])
df2['hm'] = df2['hm'].interpolate(method='linear', limit_direction='forward', x=df2[df2.corr()['hm'].drop('hm').idxmax()])
df2['ws'][df2[df2['ws'].isna()].index] = np.nanmedian(df2['ws'])
df2['rn'] = df2['rn'].interpolate(method='linear', limit_direction='forward', x=df2[df2.corr()['rn'].drop('rn').idxmax()])
df2['re'][df2[df2['re'].isna()].index] = np.nanmean(df2['re'])
df2['ts'][df2[df2['ts'].isna()].index] = np.nanmean(df2['ts'])
df2['si'] = df2['si'].interpolate(method='linear', limit_direction='forward', x=df2[df2.corr()['si'].drop('si').idxmax()])
df2['ss'] = df2['ss'].interpolate(method='linear', limit_direction='forward', x=df2[df2.corr()['ss'].drop('ss').idxmax()])
df2['sn'] = df2['sn'].interpolate(method='linear', limit_direction='forward', x=df2[df2.corr()['sn'].drop('sn').idxmax()])
df2=df2.fillna(0)

df2['mm'] = df2['mmddhh'] // 10000           # 'mmddhh' 컬럼에서 'mm', 'dd', 'hh' 분리
df2['dd'] = (df2['mmddhh'] // 100) % 100
df2['hh'] = df2['mmddhh'] % 100

In [ ]:
df2.to_csv('Feature Engineering(spring_train).csv',index=False)

In [ ]:
# spring_test 만들기 (다른 계절도 동일)

In [ ]:
# spring_train의 각 컬럼별 선택한 방법을 spring_test의 각 컬럼에도 동일하게 적용시켜주는 함수

def test(mean,median,bogan): # 변수는 컬럼 리스트로 넣을 것
  '''train에서 평균법이 최상의방법이었던 컬럼명은 첫번째 리스트로, 중위수법이 최상이었던 컬럼명은 두번째 리스트로, 보간법이 최상이었던 컬럼명은 세번째 리스트로
  예시::test(['ta','td','sn'],[],['hm','ws','rn','re','si','ss']  '''
  df = df_test.copy()
  dft = df_train.copy()
  df2 = df.replace([-99, -99.9, -999], np.nan)  # 결측치값들 nan으로 변환
  dft2 = dft.replace([-99, -99.9, -999], np.nan)

  for i in mean: # 평균법
    df2[i][df2[df2[i].isna()].index] = np.nanmean(dft2[i])

  for i in median: # 중위수
    df2[i][df2[df2[i].isna()].index] = np.nanmedian(dft2[i])

  list_bo1 = bogan # 보간법
  list_bo2 = [i+'_test' for i in list_bo1]
  for i, j in zip(list_bo2,list_bo1):
    dft2[i] = df2[j]
    df2[j] = dft2[i].interpolate(method='linear', limit_direction='forward', x=dft2[dft2.corr()[i].drop(i).idxmax()])
    dft2 = dft2.drop(i,axis=1)

  df2 = df2.fillna(0) # 나머지 결측치 0으로 변경

  df2['mm'] = df2['mmddhh'] // 10000           # 'mmddhh' 컬럼에서 'mm', 'dd', 'hh' 분리
  df2['dd'] = (df2['mmddhh'] // 100) % 100
  df2['hh'] = df2['mmddhh'] % 100
  return df2

In [ ]:
df2=test(['td','re'],['ta','ws'],['hm','rn','si','ss','sn'])#봄
df2.to_csv('Feature Engineering(spring_test).csv',index=False)

머신러닝 모델탐색

In [ ]:
# 모델탐색

In [ ]:
# AutoML로 적당한 모델 탐색
setup(data=X, target=y)
best_model = compare_models(sort='MAE')

In [ ]:
# 상위 5개 모델 직접 확인

In [ ]:
df_spring_train = pd.read_csv('/content/Feature Engineering(spring_train).csv')
df_spring_test = pd.read_csv('/content/Feature Engineering(spring_test).csv')

In [ ]:
## pre 데이터 생성
pref = df_spring_train.copy()
preg = df_spring_test.copy()

pref['stn'] = pref['stn'].astype(str)

# 1. 필요없는 컬럼들 drop
to_drops = [
    "year",
]
for to_drop in to_drops:
    if to_drop in pref.columns:
        pref = pref.drop(to_drop, axis="columns")
    if to_drop in preg.columns:
        preg = preg.drop(to_drop, axis="columns")

y = pref['ts']
pref = pref.drop('ts',axis=1)

# 2. 라벨 인코더 미리 적용, 정규화도 미리 적용하고, 변화된 부분만 다시 적용
object_cols = []
non_object_cols = []
for col in pref.columns:
    if pref[col].dtype == "object":
        object_cols.append(col)
    else:
        non_object_cols.append(col)

# LabelEncoder 적용
from sklearn.preprocessing import LabelEncoder
for col in object_cols:
    le = LabelEncoder()
    le = le.fit(pref[col])
    pref[col] = le.transform(pref[col])

    for label in np.unique(preg[col]):
        if label not in le.classes_:
            le.classes_ = np.append(le.classes_, label)
    preg[col] = le.transform(preg[col])

# StandardScaler 적용
from sklearn.preprocessing import StandardScaler
for col in non_object_cols:
    ss = StandardScaler()
    pref[col] = ss.fit_transform(pref[[col]])

    if col in preg.columns:
        preg[col] = ss.transform(preg[[col]])
X = pref

In [ ]:
x_train,x_test,y_train,y_test =train_test_split(X,y,random_state=42)

In [ ]:
#ExtraTreesRegressor
model = ExtraTreesRegressor(n_estimators=100, max_features="auto", random_state=42)
model.fit(x_train,y_train)
print(model.score(x_train,y_train),model.score(x_test,y_test))
mean_absolute_error(y_test,model.predict(x_test))

In [ ]:
# CatBoostRegressor
model = CatBoostRegressor(random_state=42)
model.fit(x_train,y_train)
print(model.score(x_train,y_train),model.score(x_test,y_test))
mean_absolute_error(y_test,model.predict(x_test))

In [ ]:
# xgboost
model = xgb.XGBRegressor(random_state=42)
model.fit(x_train,y_train)
print(model.score(x_train,y_train),model.score(x_test,y_test))
mean_absolute_error(y_test,model.predict(x_test))

In [ ]:
# LGBMRegressor
model = LGBMRegressor(random_state=42)
model.fit(x_train,y_train)
print(model.score(x_train,y_train),model.score(x_test,y_test))
mean_absolute_error(y_test,model.predict(x_test))

In [ ]:
#RandomForestRegressor
model = RandomForestRegressor(n_estimators=100, max_depth=10, random_state=42)
model.fit(x_train,y_train)
print(model.score(x_train,y_train),model.score(x_test,y_test))
mean_absolute_error(y_test,model.predict(x_test))

In [ ]:
# ExtraTreesRegressor,CatBoostRegressor,XGBRegressor로 앙상블 결정.
# ExtraTreesRegressor,CatBoostRegressor(2개) vs ExtraTreesRegressor,CatBoostRegressor,XGBRegressor(3개)

In [ ]:
# 2개 모델 앙상블

# CatBoostRegressor 모델 초기화
catboost_model = CatBoostRegressor(random_state=42)

# ExtraTreesRegressor 모델 초기화
extra_trees_model = ExtraTreesRegressor(random_state=42)

# VotingRegressor를 사용하여 앙상블 모델 초기화
ensemble_model = VotingRegressor([('catboost', catboost_model), ('extra_trees', extra_trees_model)])

ensemble_model.fit(x_train, y_train)
print(ensemble_model.score(x_train,y_train),ensemble_model.score(x_test,y_test))
mean_absolute_error(y_test,ensemble_model.predict(x_test))

In [ ]:
# 3개 모델 앙상블

# CatBoostRegressor 모델 초기화
catboost_model = CatBoostRegressor(random_state=42)

# XGBoost 모델 초기화
xgboost_model = xgb.XGBRegressor(random_state=42)

# ExtraTreesRegressor 모델 초기화
extra_trees_model = ExtraTreesRegressor(random_state=42)

# VotingRegressor를 사용하여 앙상블 모델 초기화
ensemble_model = VotingRegressor([('catboost', catboost_model), ('xgboost', xgboost_model), ('extra_trees', extra_trees_model)])

ensemble_model.fit(x_train, y_train)
print(ensemble_model.score(x_train,y_train),ensemble_model.score(x_test,y_test))
mean_absolute_error(y_test,ensemble_model.predict(x_test))

In [ ]:
# 2개 모델 앙상블 하는 것으로 결정

In [ ]:
# PolynomialFeatures로 특성을 늘리면 사계절 모두 좋은 결과를 얻는다는 것 확인

In [ ]:
pf = PolynomialFeatures(degree=2)
X_poly = pf.fit_transform(X)
x_train,x_test,y_train,y_test=train_test_split(X_poly,y,random_state=42)
# CatBoostRegressor 모델 초기화
catboost_model = CatBoostRegressor(random_state=42)

# ExtraTreesRegressor 모델 초기화
extra_trees_model = ExtraTreesRegressor(random_state=42)

# VotingRegressor를 사용하여 앙상블 모델 초기화
ensemble_model = VotingRegressor([('catboost', catboost_model), ('extra_trees', extra_trees_model)])

ensemble_model.fit(x_train, y_train)
print(ensemble_model.score(x_train,y_train),ensemble_model.score(x_test,y_test))
mean_absolute_error(y_test,ensemble_model.predict(x_test))

In [ ]:
# optuna 파라메터튜닝 + LGBM 모델

In [ ]:
pf = PolynomialFeatures(degree=2)
X_poly = pf.fit_transform(X)
x_train, x_test, y_train, y_test = train_test_split(X_poly, y, random_state=42)

In [ ]:
# Objective 함수 정의
def objective(trial):
    # 하이퍼파라미터 탐색 범위 설정
    params = {
        'num_leaves': trial.suggest_int('num_leaves', 10, 200),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
    }

    # LGBMRegressor 모델 초기화
    model = LGBMRegressor(random_state=42, **params)

    # 모델 학습
    model.fit(x_train, y_train)

    # 평가 지표 (MAE) 계산
    mae = mean_absolute_error(y_test, model.predict(x_test))

    return mae

# 데이터 분할
# x_train, x_test, y_train, y_test = train_test_split(X, y, random_state=42)

# Optuna 최적화 설정
study = optuna.create_study(direction='minimize')

# 최적화 진행 상황 출력
def print_study_statistics(study, trial):
    print(f'Trial {trial.number}/{len(study.trials)} - Loss: {trial.value:.4f} - Best Loss: {study.best_value:.4f} - Params: {trial.params}')

study.optimize(objective, n_trials=100, callbacks=[print_study_statistics])

# 최적 하이퍼파라미터 출력
print('Best Parameters:', study.best_params)
print('Best MAE:', study.best_value)

# 최적 하이퍼파라미터로 모델 재학습
best_params = study.best_params
model = LGBMRegressor(random_state=42, **best_params)
model.fit(x_train, y_train)

# 모델 성능 평가
print('Train Score:', model.score(x_train, y_train))
print('Test Score:', model.score(x_test, y_test))
print('MAE:', mean_absolute_error(y_test, model.predict(x_test)))

In [ ]:
# optuna 파라메터튜닝 + LGBM 모델의 검증점수가 기존의 앙상블 모델보다 훨씬 더 좋은 성능을 냄.

In [ ]:
# 따라서 optuna 파라메터튜닝 + LGBM 모델로 학습
pf = PolynomialFeatures(degree=2)
X_poly = pf.fit_transform(X)
model = LGBMRegressor(random_state=42,num_leaves=197,learning_rate=0.2237698754656206,max_depth=10,min_child_samples=5)
model.fit(X_poly,y)

In [ ]:
#최종 답 추출
preg_poly = pf.transform(preg)
y_pred = model.predict(preg_poly)
print(y_pred)
print(len(preg),len(df_spring_test))

In [ ]:
# 답 csv파일로 저장
pd.concat([df_spring_test, pd.DataFrame(y_pred)], axis="columns").to_csv("output_spring.csv")